# 0. 카카오 비전 API

카카오에서 최근에 흥미로운 서비스를 시작했습니다. 컴퓨터 비전(Computer Vision)으로 이미지 분석을 해주는 API를 **카카오 비전**을 공개한 것입니다. 카카오 비전은 이미지를 분석해서 콘텐츠 형태와 의미를 이해하고, 이미지를 활용할 수 있게 도와주는 API입니다. 현재 제공되는 API의 종류는 다음과 같습니다. 

- 얼굴 검출
- 상품 검출
- 썸네일 생성
- 성인 이미지 판별

얼굴 인식은 컴퓨터 비전에서 특별히 새로운 기술은 아닙니다만, 저는 카카오에서 공개한 API의 성능을 한번 확인하고자 시도해 봤습니다. 카카오 비전에 대한 자세한 것은 [이곳](https://developers.kakao.com/docs/restapi/vision)를 참고하세요.

# 1. 카카오 비전 API 준수사항

API 사용시 다음과 같은 사항을 준수해야 합니다. 

- 서버에서 호출하는 경우 REST API 키를 사용해 호출합니다.
- 모든 API는 file 또는 image_url 중 하나의 값을 지정해야 합니다.
- file에 업로드되는 이미지와 image_url에 지정되는 이미지는 png 또는 jpg 포맷만 지원합니다.


# 2. 활용 예제 살펴보기

카카오에서 간단한 `파이썬` 예제 코드를 제공하고 있습니다. 저는 거기에서 **얼굴을 추출해 모자이크 처리하는 예제**를 테스트 해보겠습니다. 작동원리를 간단하게 서술해 보면, 먼저 카카오 API로 사진상의 얼굴의 좌표를 인식하고 `pillow` 라이브러리를 이용해서 얼굴 부분의 이미지를 잘라냅니다. 그리고 리사이즈를 한뒤 원본 사진에 다시 붙여(모자이크처리) 넣습니다. 

예제로 사용할 사진은 아래와 같이 준비했습니다.

1. 원숭이 사진  

![monkey](https://i.imgur.com/HIp4WAQ.jpg)
  
  
2. 단체사진  

![peoples](https://i.imgur.com/9n0fVoI.jpg)

사용한 코드는 아래와 같습니다. 몇가지 설명을 덧붙이자면,

- 본인의 API 키를 넣어야 작동합니다.
- 변환할 사진 파일은 jpg 혹은 png 포멧이어야 합니다.
- 사진 파일의 위치는 기본적으로 같은 폴더에 있으면 바로 작동하게 만들었습니다.
- 모자이크 처리된 파일은 파일명에 *mosaic_*를 붙여서 생성됩니다.

코드를 `kakao_api.py`로 저장하고 터미널에서
```bash
python kakao_api.py -f ./.
```
라고 입력합니다.

In [1]:
import sys
import argparse
import requests
from PIL import Image, ImageFilter
import glob
import os

API_URL = 'https://kapi.kakao.com/v1/vision/face/detect'
# 아래 My Kakao api key, 본인 계정의 API 키로 변경해서 사용하세요.
MYAPP_KEY = 'b5f1dde7995c36168402318b69308ef218614'

def detect_face(filename):
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}

    try:
        files = { 'file' : open(filename, 'rb')}
        resp = requests.post(API_URL, headers=headers, files=files)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        print(str(e))
        sys.exit(0)

def mosaic(filename, detection_result):
    image = Image.open(filename)

    for face in detection_result['result']['faces']:
        x = int(face['x']*image.width)
        w = int(face['w']*image.width)
        y = int(face['y']*image.height)
        h = int(face['h']*image.height)
        box = image.crop((x,y,x+w, y+h))
        # 모자이크 강도를 조절하려면 사이즈를 조절하세요.
        box = box.resize((20,20), Image.NEAREST).resize((w,h), Image.NEAREST)
        image.paste(box, (x,y,x+w, y+h))
    return image



if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Mosaic all of faces.')
    parser.add_argument('-f', '--folder', type=str, nargs='?', default="./.",
                        help='image file to hide faces', required = True)
    args = parser.parse_args()
    # Make files lists
    files = glob.glob('*.jpg') + glob.glob('*.jpeg') + glob.glob('*.png')
    # make loop
    for i in files:
        head, tail = os.path.split(i)
        try:
            detection_result = detect_face(i)
            image = mosaic(i, detection_result)           
            image.save(head+'mosaic_'+tail,'JPEG')
            print('! Add mosaic on : ' + tail)
        except:
            print('! Face does not detected on : ' + tail) 
    
    print('!!  All process finished  !!')


!!  All process finished  !!


# 3. 결과 확인

1. 원숭이 사진  

![](https://i.imgur.com/SMNZSM4.jpg)
  
2. 단체사진  

![](https://i.imgur.com/DwBjyNp.jpg)
  
  
얼굴을 인식해서 모자이크 처리가 되는것을 볼수 있습니다. 첫 번째 원숭이 사진은 물론이고 단체사진의 얼굴도 인식을 잘 하는 것을 확인할 수 있습니다. 물론 모든 사진에서 성공적인 결과를 얻지는 못했습니다만, 얼굴 인식의 경우는 상당히 잘 작동합니다. 다른 API들도 테스트를 해보니 상품 검출 API는 아직 인식률과 오작동이 많습니다. 아마도 기술적으로 좀 더 어렵기 때문이겠죠. 테스트중에 알아낸 또 다른것은 사진 용량의 제한입니다. 사진 파일의 용량이 10MB 이상이면 에러메세지가 나오는데 아마도 제한을 걸어둔것 같습니다.